In [3]:
# A basic test of interactions with lab equipment via SCPI. Frequency will be adjusted via the PLL

import pyvisa
import time
import subprocess

# Establish connection to the power supply
rm = pyvisa.ResourceManager("@py")

psu = rm.open_resource('TCPIP::169.254.58.10::gpib0,7::INSTR')

# Need to verify that this is the power supply. psu_verification should be 
# something like "Keysight Technologies,E3646A,0,X.X-Y.Y-Z.Z"
psu_verification = psu.query("*IDN?")

psu.write("*RST")  # Reset the power supply

start_voltage = 0.5

end_voltage = 1.25

step_size = 0.05  

# Loop through the voltage range and set the voltage
current_voltage = start_voltage 

# Only channel 2 works
psu.write("INST:SEL CH2")

# Set current to 100mA
psu.write("CURR 0.1")

while current_voltage <= end_voltage: # Set the voltage   
     psu.write(f"VOLT {current_voltage}")

     voltage = psu.query("VOLT?") 
     print(f"Voltage set to: {voltage} V") 
 
     current_voltage += step_size
     time.sleep(3)

     test_num = 0
     ratio_integer = 0
     while test_num <= 10:
          ratio_fractional = 8388608
          if test_num % 2 == 0:
               ratio_fractional = 0
               ratio_integer += 1
          test_num += 1

          # Insert benchmark here to run at every step.
          # Write a benchmark that uses the PLL to adjust frequency, then have the Pico relay the frequency over to the PC.
          # Or can think of another way to retrieve the PLL CLKOUT.
     
          subprocess.run(["uart_tsi", "+tty=/dev/ttyUSB*", "+baudrate=921600", f"+init_write=0x10000070:{ratio_fractional}", f"+init_write=0x1000006C:{ratio_integer}", "build/shmoo_bench.elf"])

Error closing VISA link: socket.timeout: The instrument seems to have stopped responding.
Voltage set to: +5.00000000E-01
 V
Voltage set to: +5.50000000E-01
 V
Voltage set to: +6.00000000E-01
 V
Voltage set to: +6.50000000E-01
 V
Voltage set to: +7.00000000E-01
 V
Voltage set to: +7.50000000E-01
 V
Voltage set to: +8.00000000E-01
 V
Voltage set to: +8.50000000E-01
 V
Voltage set to: +9.00000000E-01
 V
Voltage set to: +9.50000000E-01
 V
Voltage set to: +1.00000000E+00
 V
Voltage set to: +1.05000000E+00
 V
Voltage set to: +1.10000000E+00
 V
Voltage set to: +1.15000000E+00
 V
Voltage set to: +1.20000000E+00
 V
